In [104]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy

In [105]:
# those cells are gpt generated
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

df = pd.read_csv(url, names=column_names, sep=', ', na_values='?', engine='python')

df.dropna(inplace=True)

X_raw = df.drop('income', axis=1)

y = (df['income'] == '>50K').astype(int).values

X_processed = pd.get_dummies(X_raw, drop_first=True)

X = X_processed.values

In [106]:
X = X.astype(float)
y = y.astype(float)

In [107]:
X = (X-X.mean())/X.std().view()
y = (y-y.mean())/y.std()

In [108]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [109]:
y = y.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
m = X.shape[0]
y.shape
X.shape
y = y.T
X = X.T
y_test = y_test.T
X_test = X_test.T

X.shape, y.shape, X_test.shape, y_test.shape

((96, 24129), (1, 24129), (96, 6033), (1, 6033))

In [110]:
def layer_sizes(X, y):
  n_x = X.shape[0]
  n_h1 = 50
  n_h2 = 25
  n_h3 = 10
  n_y = y.shape[0]
  return (n_x, n_h1, n_h2, n_h3, n_y)

In [111]:
layer_sizes(X, y)

(96, 50, 25, 10, 1)

In [112]:
parameters = {}

def initialize_parameters(layers_units):
  layers = len(layers_units)
  for i in range(1, layers):
    wkey = f'W{i}'
    parameters[wkey] = np.random.randn(layers_units[i], layers_units[i-1]) * 0.01
    bkey = f'b{i}'
    parameters[bkey] = np.zeros((layers_units[i], 1))
  return parameters

In [113]:
def sigmoid(Z):
  s = 1 / (1 + np.exp(-Z))
  return s

In [114]:
def relu(Z):
  return np.maximum(0, Z)

In [115]:
def leaky_relu(Z):
  return np.maximum(0.01*Z, Z)

In [116]:
def relu_derivative(Z):
    dZ = np.array(Z, copy=True)
    dZ[Z <= 0] = 0
    dZ[Z > 0] = 1
    return dZ

In [117]:
cache = {}
def forward_propagation(X, parameters):
  L = (len(parameters) // 2) + 1
  A_prev = X
  Z = 0
  for l in range(1, L):
    W = parameters[f'W{l}']
    b = parameters[f'b{l}']

    Z = np.dot(W, A_prev) + b
    A_next = relu(Z)
    A_prev = A_next
    cache[f'Z{l}'] = Z
    cache[f'A{l}'] = A_prev
  W = parameters[f'W{L-1}']
  b = parameters[f'b{L-1}']
  A_final = sigmoid(Z)
  cache[f'A{L-1}'] = A_final
  return A_final, cache

In [118]:
def compute_cost(A_final, y):
  m = y.shape[1]
  logprobs = np.sum(np.multiply(np.log(A_final), y)) + np.sum(np.multiply(np.log(1-A_final),(1-y)))
  cost = -logprobs/m
  cost = float(np.squeeze(cost))
  return cost



In [119]:
grads = {}
def backward_propagation(parameters, cache, X, Y):
  m = X.shape[1]
  L = len(parameters)//2
  cache["A0"] = X
  flag = True
  for l in range(L, 0, -1):
    A = cache[f'A{l}']
    A_prev = cache[f'A{l-1}']
    if flag:
      dZ = A - Y
      dW = np.dot(dZ, A_prev.T) / m
      db = np.sum(dZ, axis=1, keepdims=True) / m
      flag = False
    else:
      W_next = parameters[f'W{l+1}']
      Z = cache[f'Z{l}']
      dZ = np.dot(W_next.T, dZ)*relu_derivative(Z)
      dW = np.dot(dZ, A_prev.T) / m
      db = np.sum(dZ, axis=1, keepdims=True) / m
    grads[f'dW{l}'] = dW
    grads[f'db{l}'] = db
  return grads

In [120]:
def update_parameters(parameters, grads, learning_rate = 1.2):
  L = (len(parameters)//2)+1
  for l in range(1, L):
    W = copy.deepcopy(parameters[f'W{l}'])
    b = parameters[f'b{l}']
    dW = grads[f'dW{l}']
    db = grads[f'db{l}']
    W -= learning_rate * dW
    b -= learning_rate * db
    parameters[f'W{l}']=W
    parameters[f'b{l}']=b
  return parameters

In [121]:
def nn_model(X, y, layers, num_iterations = 101, print_cost=True):
  np.random.seed(3)


  parameters = initialize_parameters(layers)

  for i in range(0, num_iterations):
    A_final, cache = forward_propagation(X, parameters)
    cost = compute_cost(A_final, y)
    grads = backward_propagation(parameters, cache, X, y)
    parameters = update_parameters(parameters, grads)
    if print_cost and i % 10 == 0:
      print("Cost after iteration %i: %f" %(i, cost))

  return parameters
model = nn_model(X, y, layer_sizes(X, y))

Cost after iteration 0: 0.693148
Cost after iteration 10: 0.075222
Cost after iteration 20: 0.033449
Cost after iteration 30: 0.018501
Cost after iteration 40: 0.010650
Cost after iteration 50: 0.005710
Cost after iteration 60: 0.002255
Cost after iteration 70: -0.000335
Cost after iteration 80: -0.002371
Cost after iteration 90: -0.004031
Cost after iteration 100: -0.005421


In [122]:
model

{'W1': array([[ 0.01786917,  0.00624614,  0.00094856, ...,  0.00375679,
         -0.00576325, -0.00111071],
        [ 0.00680049, -0.00925962, -0.00299212, ..., -0.00559373,
         -0.00587501, -0.01545555],
        [-0.0012719 ,  0.00117535,  0.00446352, ...,  0.02316429,
         -0.00207713, -0.00014403],
        ...,
        [-0.00353703,  0.00136472,  0.01385877, ..., -0.0137996 ,
         -0.00092806,  0.00054963],
        [ 0.00941324,  0.01197703,  0.00740184, ..., -0.00755911,
         -0.0055821 ,  0.00246372],
        [-0.01111776,  0.00684055,  0.00556659, ...,  0.00506732,
          0.00021078,  0.0079964 ]]),
 'b1': array([[ 1.79431351e-04],
        [-1.11102282e-04],
        [-6.33475211e-05],
        [ 1.98782079e-04],
        [-2.38416008e-06],
        [-3.36357430e-04],
        [-2.36771670e-05],
        [ 1.12589979e-04],
        [-7.01485618e-06],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [-7.33163183e-05],
        [ 1.26302863e-04],
        [ 

In [123]:
def predict(parameters, X_test):
  A2, cache = forward_propagation(X_test, parameters)
  predictions = (A2>=0.5).astype(int)
  return predictions

In [124]:
parameters = model
predictions = predict(parameters, X_test)


In [125]:
print ('Accuracy: %d' % float((np.dot(y_test, predictions.T) + np.dot(1 - y_test, 1 - predictions.T)) / float(y_test.size) * 100) + '%')

Accuracy: 99%


/tmp/ipython-input-3541431280.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print ('Accuracy: %d' % float((np.dot(y_test, predictions.T) + np.dot(1 - y_test, 1 - predictions.T)) / float(y_test.size) * 100) + '%')
